# Assignment: OECD Producer Price Index

The [producer price index](https://en.wikipedia.org/wiki/Producer_price_index) (PPI) measures the rate of change of price for products sold as they leave the producer. [OECD](http://oecd.org/), an intergovernmental organization, maintains a dataset of PPI for countries around the world. In this assignment, you will visualize the PPI of various countries from Jan 2011 to Jan 2023 as high dimensional data.

## Data

* [PPI dataset](https://data.oecd.org/price/producer-price-indices-ppi.htm#indicator-chart)

The important columns of this dataset are `LOCATION`, `TIME` and `Value`. We will treat the per-country PPI values over time as a single data point. I.e. Each high dimension data point consists of all the values from Jan 2011 to Jan 2023 for a given country. You may want to use `pandas.pivot` to switch the data frame from long form to wide form. For this assignment, we will replace all `NaN` values by 0. 

## Task

Your task for this assignment is to find a two-dimensional embedding of this high dimensional dataset that clusters countries with similar PPI value history together. The final visualization should be a 2D scatter plot. The x and y axis should map to the components computed from the dimension reduction algorithm.  The location information should be encoded as color.

Please use this notebook for this assignment.

In [1]:
import altair as alt
import pandas as pd
import sklearn

url = "https://github.com/qnzhou/practical_data_visualization_in_python/files/14559866/oecd_ppi.csv"
data = pd.read_csv(url)

In [2]:
# Let's look at the shape of the data
print(data.shape)

# Checkout a few rows...
data.head()

(5751, 8)


,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
0,AUT,PPI,DOMESTIC,IDX2015,M,2011-01,98.65053,NaN
1,AUT,PPI,DOMESTIC,IDX2015,M,2011-02,99.12756,NaN
2,AUT,PPI,DOMESTIC,IDX2015,M,2011-03,99.98622,NaN
3,AUT,PPI,DOMESTIC,IDX2015,M,2011-04,100.36780,NaN
4,AUT,PPI,DOMESTIC,IDX2015,M,2011-05,100.36780,NaN


In [3]:
# Now let's pivot the data from long form to wide form while isolating the important columns
important_data = data.pivot(index='LOCATION', columns='TIME', values='Value')

# Let's look at the shape of the pivot data
print(important_data.shape)

# Make sure it we pivot correctly...
important_data.head()

(40, 145)


TIME,2011-01,2011-02,2011-03,2011-04,2011-05,2011-06,2011-07,2011-08,2011-09,2011-10,...,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01
LOCATION,,,,,,,,,,,,,,,,,,,,,
AUT,98.65053,99.12756,99.98622,100.36780,100.36780,100.46330,100.46330,100.36780,100.46330,100.55870,...,125.3637,126.6903,128.1275,128.0169,127.2430,128.7908,127.9063,126.4692,127.1325,NaN
BEL,105.10000,106.20000,107.10000,107.90000,107.70000,107.60000,107.60000,107.30000,107.50000,107.40000,...,155.0000,157.3000,160.2000,158.2000,157.7000,156.2000,158.1000,157.1000,155.3000,NaN
CHE,105.01240,105.27230,105.64670,105.65280,105.47970,105.13020,104.75290,104.40430,104.50530,104.24700,...,107.2716,108.3059,108.9728,109.3016,109.6301,109.7087,109.6758,109.7475,109.4631,NaN
COL,89.16204,90.01339,90.61604,90.84561,91.39085,91.40999,91.10388,91.38129,91.48651,92.16568,...,143.0170,145.6189,145.6189,149.2730,148.4886,149.8948,152.2862,154.2950,153.5393,NaN
CRI,84.55310,85.49834,87.08256,88.83438,89.42461,90.18393,90.41148,90.10818,89.99454,90.05784,...,131.6622,134.6859,137.2771,138.3730,137.4205,136.5310,136.1210,135.5293,134.7552,NaN


In [4]:
# Clean the data by replacing NAN values with 0
important_data = important_data.fillna(0)

# Make sure we replaced correctly...
important_data.head()

TIME,2011-01,2011-02,2011-03,2011-04,2011-05,2011-06,2011-07,2011-08,2011-09,2011-10,...,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01
LOCATION,,,,,,,,,,,,,,,,,,,,,
AUT,98.65053,99.12756,99.98622,100.36780,100.36780,100.46330,100.46330,100.36780,100.46330,100.55870,...,125.3637,126.6903,128.1275,128.0169,127.2430,128.7908,127.9063,126.4692,127.1325,0.0
BEL,105.10000,106.20000,107.10000,107.90000,107.70000,107.60000,107.60000,107.30000,107.50000,107.40000,...,155.0000,157.3000,160.2000,158.2000,157.7000,156.2000,158.1000,157.1000,155.3000,0.0
CHE,105.01240,105.27230,105.64670,105.65280,105.47970,105.13020,104.75290,104.40430,104.50530,104.24700,...,107.2716,108.3059,108.9728,109.3016,109.6301,109.7087,109.6758,109.7475,109.4631,0.0
COL,89.16204,90.01339,90.61604,90.84561,91.39085,91.40999,91.10388,91.38129,91.48651,92.16568,...,143.0170,145.6189,145.6189,149.2730,148.4886,149.8948,152.2862,154.2950,153.5393,0.0
CRI,84.55310,85.49834,87.08256,88.83438,89.42461,90.18393,90.41148,90.10818,89.99454,90.05784,...,131.6622,134.6859,137.2771,138.3730,137.4205,136.5310,136.1210,135.5293,134.7552,0.0


In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Compute PCA on data
data_scaled = StandardScaler().fit_transform(important_data)
pca = PCA(2)
r = pca.fit_transform(data_scaled)

In [6]:
# Define PCA dataframe
df_pca = pd.DataFrame({
    'x':r[:,0], 
    'y':r[:,1], 
    'label':list(important_data.index),
})

# Plot the PCA data
chart = alt.Chart(df_pca).mark_point(size=50).encode(
    x=alt.X('x:Q', title='Principle Component 1'), 
    y=alt.Y('y:Q', title='Principle Component 2'), 
    color=alt.Color('label:N').legend(title='Country', symbolLimit=len(important_data))
    ).properties(title='2D PCA of OECD PPI by Country', width=800, height=600)

chart.save('chart.png')
chart

alt.Chart(...)